# AgentScope example connecting to Sap Generative AI Hub via LiteLLM

## [How AgentScope works](https://doc.agentscope.io/tutorial/)

## LLM access via LiteLLM Proxy
AgentScope supports the [LiteLLM Proxy](https://docs.litellm.ai/docs/simple_proxy) for access via OpenAI API calls.

Follow details in [LiteLLM Proxy setup for SAP Gen. AI Hub](../proxy_set_up.ipynb)

## Installation

In [1]:
%pip install agentscope litellm

  Using cached python_datauri-3.0.2-py2.py3-none-any.whl.metadata (3.3 kB)
  Using cached shortuuid-1.0.13-py3-none-any.whl.metadata (5.8 kB)
  Using cached sounddevice-0.5.3-py3-none-macosx_10_6_x86_64.macosx_10_6_universal2.whl.metadata (1.6 kB)
  Using cached opentelemetry_sdk-1.39.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached cached_property-2.0.1-py3-none-any.whl.metadata (10 kB)
  Using cached bidict-0.23.1-py3-none-any.whl.metadata (8.7 kB)
  Using cached python_engineio-4.12.3-py3-none-any.whl.metadata (2.2 kB)
  Using cached simple_websocket-1.1.0-py3-none-any.whl.metadata (1.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.4 MB/s eta 0:00:00
Using cached python_datauri-3.0.2-py2.py3-none-any.whl (5.8 kB)
Using cached shortuuid-1.0.13-py3-none-any.whl (10 kB)
Using cached sounddevice-0.5.3-py3-none-macosx_10_6_x86_64.macosx_10_6_universal2.whl (108 kB)
Using cached bidict-0.23.1-py3-none-any.whl (32 kB)
Using cached python_engineio-4.12.3-py3-none-any.whl (

## Set env variables
Add the following variables from the service key in a file called ".env" and put it in the same folder where you run the notebook:
```
LITELLM_PROXY_API_KEY=sk-1234
PROXY_BASE_URL=http://localhost:4000
```

## Run the AgentScope with LiteLLM and SAP LLMs

In [2]:
import asyncio
import os

import litellm
from agentscope.agent import ReActAgent
from agentscope.formatter import DashScopeChatFormatter
from agentscope.memory import InMemoryMemory
from agentscope.message import Msg, TextBlock
from agentscope.model import OpenAIChatModel
from agentscope.tool import Toolkit, ToolResponse
from dotenv import load_dotenv

Load your credentials as environment variables.

In [3]:
litellm.use_litellm_proxy = True
load_dotenv()
api_base = os.getenv("PROXY_BASE_URL")
api_key = os.getenv("LITELLM_PROXY_API_KEY")

Set up the model with your proxy params

In [4]:
sap_model = OpenAIChatModel(model_name='sap/gpt-4o',
                            api_key=api_key,
                            client_args={"base_url": api_base},
                            stream=False)

Define the agent tools with a properly formatted docstring. Including the Args section is necessary for the correct functioning of the tool. Also the function must return its response as a ToolResponse object.

In [5]:
def get_weather(city: str) -> ToolResponse:
    """Retrieves the current weather report for a specified city.
    Args:
        city (str): The name of the city to retrieve weather information for.
            Examples: "New York", "London", "Tokyo".
    """
    city_normalized = city.lower().replace(" ", "")

    mock_weather_db = {
        "newyork": "The weather in New York is sunny with a temperature of 25°C.",
        "london": "It's cloudy in London with a temperature of 15°C.",
        "tokyo": "Tokyo is experiencing light rain and a temperature of 18°C.",
    }

    if city_normalized in mock_weather_db:
        return ToolResponse(content=[
            TextBlock(type="text",
                      text=mock_weather_db[city_normalized])
        ])
    else:
        return ToolResponse(content=[
            TextBlock(type="text",
                      text=f"The weather in {city} is sunny with a temperature of 20°C.")
        ])

Register the tools in a toolkit

In [6]:
toolkit = Toolkit()
toolkit.register_tool_function(get_weather)

Define the Agent with the SAP LLM and the tool, memory and formatter objects.

In [7]:
agent = ReActAgent(
        name="weather agent",
        sys_prompt="You are a helpful weather assistant. "
                "When the user asks for the weather in a specific city, "
                "use the 'get_weather' tool to find the information. "
                "If the tool returns an error, inform the user politely. "
                "If the tool is successful, write a couple sentences for "
                "TV weather report in the city, that will be include small jok",
        model=sap_model,
        formatter=DashScopeChatFormatter(),
        toolkit=toolkit,
        memory=InMemoryMemory(),
    )

Create a message

In [8]:
msg = Msg(
        name="user",
        content="What is the weather like in Tbilisi?",
        role="user",
    )

Run agent inside the async function

In [9]:
async def run_conversation():
    result = await agent(msg)
    print(result)

Run the conversation function

In [10]:
await run_conversation()

weather agent: {
    "type": "tool_use",
    "id": "call_Cwv6sNtrDRMOoPrZj9nHFD9Q",
    "name": "get_weather",
    "input": {
        "city": "Tbilisi"
    }
}
system: {
    "type": "tool_result",
    "id": "call_Cwv6sNtrDRMOoPrZj9nHFD9Q",
    "name": "get_weather",
    "output": [
        {
            "type": "text",
            "text": "The weather in Tbilisi is sunny with a temperature of 20°C."
        }
    ]
}
weather agent: Good morning, Tbilisi! It's a beautiful sunny day with a temperature of 20°C. Perfect weather to enjoy a stroll down Rustaveli Avenue! Just remember, while the sun's out, maybe leave your umbrella behind... but don't forget your sunglasses!
Msg(id='NmedmCrW7LXh4qdc887Qfo', name='weather agent', content=[{'type': 'text', 'text': "Good morning, Tbilisi! It's a beautiful sunny day with a temperature of 20°C. Perfect weather to enjoy a stroll down Rustaveli Avenue! Just remember, while the sun's out, maybe leave your umbrella behind... but don't forget your sung